# Part 2: Streaming application using Spark Structured Streaming

1. Write code to create a SparkSession with the following requirements: 1) use four cores with a proper application name; 2) Melbourne timezone; 3) a checkpoint location has been set.


In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql import functions as F
from pyspark.sql.types import *


from pyspark.sql import SparkSession

# Import SparkConf class into the program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[4]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Assignment 2B"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Import SparkContext and SparkSession classes
from pyspark import SparkContext  # Spark
from pyspark.sql import SparkSession  # Spark SQL

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

# Set the time zone for Spark SQL session
spark.conf.set("spark.sql.session.timeZone", "Australia/Melbourne")


spark.sparkContext.setCheckpointDir("checkpoint/")

2. Similar to assignment 2A, write code to define the data schema for the data files, following the data types suggested in the metadata file. Load the static datasets(previous_application_static and value_dict) into data frames. (You can reuse your code from 2A.)


In [9]:
# Read files into dataframes - value_dict and previous_application_static
df_value_dict = spark.read.csv('value_dict.csv', header=True)
df_previous_application_static = spark.read.csv('previous_application_static.csv', header=True)
df_application_data_stream = spark.read.csv('application_data_stream.csv', header=True)

In [10]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

# Define schema for previous_application_static
previous_application_static_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("id_app", IntegerType(), True),
    StructField("contract_type", StringType(), True),
    StructField("amt_annuity", DoubleType(), True),
    StructField("amt_application", DoubleType(), True),
    StructField("amt_credit", DoubleType(), True),
    StructField("amt_down_payment", DoubleType(), True),
    StructField("amt_goods_price", DoubleType(), True),
    StructField("hour_appr_process_start", IntegerType(), True),
    StructField("rate_down_payment", DoubleType(), True),
    StructField("rate_interest_primary", DoubleType(), True),
    StructField("rate_interest_privilege", DoubleType(), True),
    StructField("name_cash_loan_purpose", StringType(), True),
    StructField("name_contract_status", StringType(), True),
    StructField("days_decision", IntegerType(), True),
    StructField("name_payment_type", StringType(), True),
    StructField("code_reject_reason", StringType(), True),
    StructField("name_type_suite", StringType(), True),
    StructField("name_client_type", StringType(), True),
    StructField("name_goods_category", StringType(), True),
    StructField("name_portfolio", StringType(), True),
    StructField("name_product_type", StringType(), True),
    StructField("channel_type", StringType(), True),
    StructField("sellerplace_area", StringType(), True),
    StructField("name_seller_industry", StringType(), True),
    StructField("cnt_payment", IntegerType(), True),
    StructField("name_yield_group", StringType(), True),
    StructField("product_combination", StringType(), True),
    StructField("days_first_drawing", IntegerType(), True),
    StructField("days_first_due", IntegerType(), True),
    StructField("days_last_due_1st_version", IntegerType(), True),
    StructField("days_last_due", IntegerType(), True),
    StructField("days_termination", IntegerType(), True),
    StructField("nflag_insured_on_approval", IntegerType(), True)
])

# Define schema for value_dict_schema
value_dict_schema = StructType([
    StructField("id", IntegerType(), True),  # Number column
    StructField("category", StringType(), True),  # String type
    StructField("key", StringType(), True),  # String type
    StructField("value", IntegerType(), True)  # Number column
])

In [56]:
df_previous_application_static.printSchema()
df_value_dict.printSchema()

root
 |-- id_app: string (nullable = true)
 |-- contract_type: string (nullable = true)
 |-- amt_annuity: string (nullable = true)
 |-- amt_application: string (nullable = true)
 |-- amt_credit: string (nullable = true)
 |-- amt_down_payment: string (nullable = true)
 |-- amt_goods_price: string (nullable = true)
 |-- hour_appr_process_start: string (nullable = true)
 |-- rate_down_payment: string (nullable = true)
 |-- rate_interest_primary: string (nullable = true)
 |-- rate_interest_privileged: string (nullable = true)
 |-- name_cash_loan_purpose: string (nullable = true)
 |-- name_contract_status: string (nullable = true)
 |-- days_decision: string (nullable = true)
 |-- name_payment_type: string (nullable = true)
 |-- code_rejection_reason: string (nullable = true)
 |-- name_type_suite: string (nullable = true)
 |-- name_client_type: string (nullable = true)
 |-- name_goods_category: string (nullable = true)
 |-- name_portfolio: string (nullable = true)
 |-- name_product_type: str

3. Using the Kafka topic from the producer in Task 1, read the streaming data with Spark Streaming, assuming all data comes in the String format. Except for the 'ts' column, you shall receive it as an Int type.



In [11]:
#configuration
hostip = "kafka"
topic = 'application_data_stream'

#Read stream data VARIABLE from Kafka PRODUCER
stream_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", f"{hostip}:9092") \
    .option("subscribe", topic) \
    .option("startingOffsets", "latest") \
    .load()

In [12]:
stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [13]:
# Convert the value column from binary to string
stream_df = stream_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "topic", "partition", "offset", "timestamp", "timestampType")

In [14]:
# Start running the query that prints the running counts to the console
query = stream_df \
    .writeStream \
    .outputMode("Append")\
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .start()


4. Then, transform the streaming data format into proper types following the metadata file schema, similar to assignment 2A. Perform the following tasks:  
a) For the 'ts' column, convert it to the timestamp format, we will use it as event_time.  
b) If the data is late for more than 1 minute, discard it.




In [16]:
from pyspark.sql.functions import col, expr, from_json
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, BinaryType, LongType

streaming_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("ts", StringType(), True),  # Assuming ts is a string representing timestamp
    StructField("key", BinaryType(), True),
    StructField("value", BinaryType(), True),
    StructField("topic", StringType(), True),
    StructField("partition", IntegerType(), True),
    StructField("offset", LongType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("timestampType", IntegerType(), True)
])

# Convert the value column from binary to string and parse JSON
parse_stream_dataframe = stream_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), streaming_schema).alias("data")) \
    .select("data.*")

# Convert 'ts' column to timestamp format
parsed_stream_df = parse_stream_dataframe.withColumn("ts", expr("CAST(ts AS TIMESTAMP)"))

# Define event_time based on 'ts' column
parsed_stream_df = parsed_stream_df.withColumn("event_time", col("ts"))

# Define watermark for event_time with a delay threshold of 1 minute
watermarking_df = parsed_stream_df \
    .withWatermark("event_time", "1 minute")

# Filter late data
filtered_df = watermarking_df.filter(col("event_time").isNotNull())

# Start running the query
query = filtered_df \
    .writeStream \
    .outputMode("Append") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .start()

5. Join the static data frames with the streaming data frame, perform data type/column conversion according to your ML model and print out the Schema. (Again, you can reuse code from 2A).


In [31]:
#join the static dataframe with the streaming dataframe

joined_df = df_value_dict.join(df_previous_application_static, df_value_dict.id== df_previous_application_static.id, "left_outer")

6. Load your ML model, and use the model and Spark to perform the following:  
    a) Every 10 seconds, print the total number of applications and number of potential default applications (prediction = 1) in the last 1 minute.  
    b) Every 15 seconds, show the total requested credit (sum of credit where default=0) in the last 15 seconds.  
    c) Every 1 minute, show the top 10 largest loan applications with the potential of default.  

In [ ]:
# 6a
from pyspark.sql import functions as F

# Defined variable window_duration_1min and window_duration_15sec as per the question for 1 minutes and 15 seconds
window_duration_1min = "1 minute"
window_duration_15sec = "15 seconds"

#Every 10 seconds, print the total number of applications and number of potential default applications (prediction = 1) in the last 1 minute.
#since i dont have ml model 
applications_counts_query = stream_df \
    .groupBy(F.window(col("timestamp"), window_duration_1min)) \
    .agg(F.sum(col("target")).alias("potential_defaults"),
         F.count("id_app").alias("total_applications")) \
    .writeStream \
    .trigger(processingTime="10 seconds") \
    .outputMode("complete") \
    .format("console") \
    .start()

In [ ]:
# 6b
# Every 15 seconds, show the total requested credit (sum of credit where default=0) in the last 15 seconds.
requested_credit_query = stream_df \
    .where(col("target") == 0) \
    .groupBy(window(col("timestamp"), window_duration_15sec)) \
    .agg(sum(col("amt_credit")).alias("total_requested_credit")) \
    .writeStream \
    .trigger(processingTime="15 seconds") \
    .outputMode("complete") \
    .format("console") \
    .start()

In [ ]:
# 6c
#Every 1 minute, show the top 10 largest loan applications with the potential of default.
top_defaults_query = stream_df \
    .where(col("target") == 1) \
    .orderBy(col("amt_credit").desc()) \
    .limit(10) \
    .writeStream \
    .trigger(processingTime="1 minute") \
    .outputMode("complete") \
    .format("console") \
    .start()

7. Write a Parquet file to store the following data:  
    a) Persist the raw data: Persist your prediction results along with application data and event_time in Parquet format; after that, read the Parquet file and show the first 10 records.  
    b) Persist the 15-second requested credit (6b) in another parquet file.


In [25]:
# Write into parquet files the unsuccessful requests partitioned by status code
query_file_sink = stream_df.writeStream.format("parquet")\
        .outputMode("append")\
        .option("path", "parquet/application_data_stream_df")\
        .option("checkpointLocation", "parquet/application_data_stream_df/checkpoint")\
        .start()

In [29]:
# Read the saved parquet data
query_file_sink_df = spark.read.parquet("parquet/application_data_stream_df")
query_file_sink_df.printSchema()
query_file_sink_df.show()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



8. Read the two parquet files from 7a and 7b as a data stream, and send the records to two topics with appropriate names.  
(Note: You shall read the parquet files as a streaming data frame and send messages to the Kafka topic when new data appears in the parquet files. The parquet files serve as an intermediate storage in this use case.)

In [ ]:
# Read Parquet files as streaming DataFrames
raw_data_stream_df = spark.readStream \
    .format("parquet") \
    .load("parquet/application_data_stream_df")


# Send records to Kafka topics
raw_data_stream_df.selectExpr("CAST(application_data AS STRING)") \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("topic", topic_7a) \
    .start()

# Reference

http://127.0.0.1:5202/notebooks/Lab/Lab%2010/1.%20Spark%20Streaming%20-%20ClickStream-Analysis.ipynb - Lab 10

http://127.0.0.1:5202/notebooks/Lab/Lab%2010/1.%20Spark%20Streaming%20-%20ClickStream-Analysis.ipynb - Lab 10

http://127.0.0.1:5202/notebooks/Lab/Lab%2011/1.%20Spark%20Streaming%20Watermarking%20DEMO.ipynb - parquet